In [ ]:
from datetime import datetime, timezone, timedelta
!python -m pip install matplotlib confluent-kafka


In [ ]:
import functools

@functools.cache
def read_ccloud_config(config_file='client.properties'):
    conf = {}
    with open(config_file) as fh:
        for line in fh:
            line = line.strip()
            if len(line) != 0 and line[0] != "#":
                parameter, value = line.strip().split('=', 1)
                conf[parameter] = value.strip()
    return conf
def read_ccloud_producer_config(config_file='client.properties'):
    conf = read_ccloud_config(config_file)
    omitted_fields = set(['schema.registry.url', 'basic.auth.credentials.source', 'basic.auth.user.info'])
    omitted_prefix = 'confluent'
    for fld in list(conf.keys()):
        if fld in omitted_fields or fld.startswith(omitted_prefix):
            conf.pop(fld, None)
    return conf



In [ ]:
# Load Generation
msg_size_bytes = 1024
msg_rate_per_s = 100
msg_send_duration_s = 180
num_producers = 20 

# Cluster Limits from https://docs.confluent.io/cloud/current/clusters/cluster-types.html#types-basic-limits-per-cluster
# on 12/1/2023
confluent_max = {}
confluent_max['active_connection_count'] = 1000
confluent_max['received_bytes'] = 250 * 1024 * 1024
confluent_max['sent_bytes'] = 750 * 1024 * 1024
confluent_max['request_count'] = 15000

#### Producer Send

In [ ]:
def delivery_callback(err, msg):
    if err:
        print('ERROR: Message failed delivery: {}'.format(err))
    else:
        #print("Produced event to topic {topic}: key = {key:12} value = {value:12}".format(
        #    topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))
        pass

# Produce Messages
from confluent_kafka import Producer
from time import sleep
import json
from random import uniform, randint

def getMessages(numMessages):
    # len is 64
    base_msg = "Upon our honor, we will monitor our data streaming application. "
    for i in range(numMessages):
        yield { 'key': f"mt_key_{randint(1,6)}", 'value': f"{base_msg * (msg_size_bytes//len(base_msg))}" }

# Simulating extra connections
producers = []
for i in range(num_producers):
    producers.append(Producer(read_ccloud_producer_config()))
    print("p",end="")
print()

startTime = datetime.now(timezone.utc)
msgSentCount = 0
for msg in getMessages(msg_rate_per_s * msg_send_duration_s):
    ts = datetime.now(timezone.utc)
    ts_str = ts.isoformat()
    msg['value'] = '{ "payload": "' + msg['value'] + '", "ts": "' + ts_str + '" }'
    producers[randint(0,num_producers-1)].produce("sale_records", key=msg['key'], value=msg['value'], callback=delivery_callback)
    msgSentCount += 1
    if not msgSentCount % 1000:
        print(msgSentCount)
        elapsed_seconds = (ts - startTime).total_seconds()
        if msgSentCount >=  elapsed_seconds * msg_rate_per_s:
            sleep(msgSentCount/msg_rate_per_s - elapsed_seconds)
for producer in producers:
    producer.flush()
endTime = datetime.now(timezone.utc)
print()

In [ ]:
import urllib.request
import json
MetricsQueries = {
    'received_bytes': {
        'title': 'Ingress',
        'yaxis': 'Bytes',
        'query': {
            'aggregations': [{ 'metric': 'io.confluent.kafka.server/received_bytes'}],
            'group_by': ['metric.topic']
        }
    },
    'sent_bytes': {
        'title': 'Egress',
        'yaxis': 'Bytes',
        'query': {
            'aggregations': [{ 'metric': 'io.confluent.kafka.server/sent_bytes'}],
            'group_by': ['metric.topic']
        }
    },
    'active_connection_count' : {
        'title': 'Active Connection Count',
        'yaxis': 'Count',
        'query': {
            "aggregations":[{ "metric":"io.confluent.kafka.server/active_connection_count"}]
        }
    },
    'request_count': {
        'title': 'Request Count',
        'yaxis': 'Count',
        'query': {
            'aggregations': [{ 'metric': 'io.confluent.kafka.server/request_count'}]
        }
    },
    'cluster_load': {
        'title': 'Cluster Load',
        'yaxis': '% Load'
    }
}
def getMetrics(startTime, endTime):
    conf = read_ccloud_config()
    url = conf['confluent.metrics.endpoint']
    headers = {
        'Authorization': f"Basic {conf['confluent.cloud_api_token']}",
        'Content-Type': 'application/json'
    }
    common = {
        "filter":{"op":"OR","filters":[{"field":"resource.kafka.id","op":"EQ","value":"lkc-v1jq15"}]},
        "granularity":"PT1M",
        "limit":1000
    }
    interval = {
        "intervals":[f"{startTime.isoformat(timespec='seconds')}/{(endTime+timedelta(minutes=1)).isoformat(timespec='seconds')}"],
    }

    responses = {}
    for qry in MetricsQueries:
        if 'query' not in MetricsQueries[qry]:
            continue
        data = MetricsQueries[qry]['query'] | common | interval

        req = urllib.request.Request(url, json.dumps(data).encode('utf-8'), headers)
        resp = urllib.request.urlopen(req)
        if resp.getcode() == 200:
            responses[qry] = json.loads(resp.read())
        else:
            print("Error: {resp.getcode()}")

    return responses
# Get Data
results = getMetrics(startTime, endTime)
cluster_load_data = []
# pick any metric to get number of entries since we aggregate per minute
for i in range(len(results['active_connection_count']['data'])):
     cl_pct = 0
     cl_ts = ""
     for metric in MetricsQueries:
        if 'query' not in MetricsQueries[metric] or i >= len(results[metric]['data']):
             continue
        cl_pct = max(cl_pct, 100.0 * results[metric]['data'][i]['value'] / confluent_max[metric])
        cl_ts = max(cl_ts, results[metric]['data'][i]['timestamp'])
     cluster_load_data.append({ 'timestamp': cl_ts, 'value': cl_pct })
results['cluster_load'] = { 'data': cluster_load_data }


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from math import sqrt, ceil


def addPlot(axs, row, col, data, title, yAxis):
    x_values = [(datetime.fromisoformat(el['timestamp']) - startTime).total_seconds() for el in data]
    y_values = [el['value'] for el in data]
    axs[row,col].plot(x_values, y_values)

    # Add labels and title
    axs[row,col].set_xlabel('Time (s)')
    axs[row,col].set_ylabel(yAxis.title())
    axs[row,col].set_title(title.title())

# Display chart but give the metrics a chance to get done
# for the ever eager run all in notebook fans
timeSinceProduce = (datetime.now(timezone.utc) - endTime).total_seconds()
if  timeSinceProduce < 120:
    sleep(120-timeSinceProduce)
    
index = 0
total_plots = len(results)
ncols = 2
nrows = ceil(total_plots/2)
fig, axs = plt.subplots(nrows,ncols,sharex=True)
for qry in results:
    addPlot(axs, index//ncols, index%ncols, results[qry]['data'],  
            title=MetricsQueries[qry]['title'], yAxis=MetricsQueries[qry]['yaxis'])
    index += 1

fig.tight_layout()
plt.show()